In [11]:
#Credit: https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from sklearn.cross_validation import train_test_split
from collections import defaultdict
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
from sklearn.feature_extraction import text 
import re
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
stopwords=text.ENGLISH_STOP_WORDS

DATAFILEPATH='/Users/Dilip_MBP/Documents/CS109/PROJECT/data/'

[nltk_data] Downloading package punkt to /Users/Dilip_MBP/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [25]:
# Read and store csv data in to Master index

bike_masterdf = pd.read_csv(DATAFILEPATH+'bike_details.csv',encoding='utf-8')
print "Number of samples in the database:", bike_masterdf.shape[0]
bike_masterdf.head(2)

Number of samples in the database: 59619


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch
0,0,50088,Schwinn Gateway,absent,Schwinn,Gateway,NaN,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-07-06 22:27:01,2015-11-03 05:36:54,https://bikeindex.org/bikes/50088,https://bikeindex.org/api/v1/bikes/50088,117,NaN,NaN,NaN,NaN,True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1436221621,1446529014,0
1,1,60140,2016 Specialized Diverge Elite DSW,WSBC601010269K,Specialized,Diverge Elite DSW,2016,NaN,NaN,False,False,NaN,1970-01-01 00:00:00,2015-10-08 19:18:50,2015-11-03 05:34:30,https://bikeindex.org/bikes/60140,https://bikeindex.org/api/v1/bikes/60140,307,NaN,NaN,52cm,NaN,True,NaN,Bike,False,NaN,NaN,NaN,aluminum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1444331930,1446528870,0


In [27]:
# Remove nulls in theft description - only stolen bikes have theft description -> store in filtered dataframe
num_stolen = len(bike_masterdf.stolen[bike_masterdf.stolen == True])
print "Number of bikes reported as stolen: ", num_stolen
print "Proportion of bikes reported as stolen:  %.2f" % (float(num_stolen)/float(bike_masterdf.shape[0]))
desc_mask = bike_masterdf.theft_description.isnull()
bike_subdf = bike_masterdf[~desc_mask]
print "Number of Stolen bikes with theft description:", bike_subdf.shape[0]

bike_subdf.to_csv(DATAFILEPATH+'stolen_bikes.csv',encoding='utf-8')
bike_subdf.head(2)

Number of bikes reported as stolen:  33182
Proportion of bikes reported as stolen:  0.56
Number of Stolen bikes with theft description: 29838


,Unnamed: 0,id,title,serial,manufacturer_name,frame_model,year,thumb,large_img,is_stock_img,stolen,stolen_location,date_stolen,registration_created_at,registration_updated_at,url,api_url,manufacturer_id,paint_description,name,frame_size,description,rear_tire_narrow,front_tire_narrow,type_of_cycle,test_bike,rear_wheel_size_iso_bsd,front_wheel_size_iso_bsd,handlebar_type_slug,frame_material_slug,front_gear_type_slug,rear_gear_type_slug,stolen_id,s_date_stolen,s_location,latitude,longitude,theft_description,locking_description,lock_defeat_description,police_report_number,police_report_department,s_rec_created_at,create_open311,sder_formatted_address,sder_street_number,sder_route,sder_postal_code,sder_neighborhood,sder_city,sder_county,sder_state,sder_country,date_stolen_epoch,registration_created_at_epoch,registration_updated_at_epoch,s_rec_created_at_epoch
3,3,11542,2013 Trek 4300,WTU124C5170G,Trek,4300,2013,NaN,NaN,False,True,"San Jose, CA, 95126",2013-06-10 06:00:00,2013-08-20 06:00:00,2015-11-03 05:11:14,https://bikeindex.org/bikes/11542,https://bikeindex.org/api/v1/bikes/11542,47,NaN,NaN,NaN,"Trek 4300, Mountain bicycle, disc brakes, blac...",True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,7889,1370844000,"San Jose, CA, 95126",37.329012,-121.916021,bike rack,NaN,NaN,T13006853,San Jose,2014-06-14 17:52:49,False,"1323 Martin Ave, San Jose, CA 95126, USA",1323,Martin Avenue,95126,Shasta-Hanchett Park,San Jose,Santa Clara County,California,United States,1370844000,1376978400,1446527474,1402768369
4,4,13359,2010 Schwinn Katana,absent,Schwinn,Katana,2010,https://files.bikeindex.org/uploads/Pu/4903/sm...,https://files.bikeindex.org/uploads/Pu/4903/la...,False,True,"Sandusky, OH, 44870",2010-07-10 06:00:00,2010-07-11 06:00:00,2015-11-03 05:17:53,https://bikeindex.org/bikes/13359,https://bikeindex.org/api/v1/bikes/13359,117,NaN,NaN,58cm,"The bike is a grey, blue, and black Schwinn Ka...",True,NaN,Bike,False,NaN,NaN,NaN,NaN,NaN,NaN,9705,1278741600,"Sandusky, OH, 44870",41.440356,-82.768114,car bike rack Reward: $75,NaN,NaN,2010004433,Sandusky,2014-06-14 18:17:08,False,"4614 Venice Heights Blvd, Sandusky, OH 44870, USA",4614,Venice Heights Boulevard,44870,NaN,Sandusky,Erie County,Ohio,United States,1278741600,1278828000,1446527873,1402769828


In [34]:
# Create function to parse incoming text, tokenize and check for  ******* Nouns and Proper Nouns
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
regex3=re.compile(r"\\\\")
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
stemmer = SnowballStemmer("english")

#use nltk tokenizer to tokenize and stem words in incoming sentence 
def clean_and_tokenize(intext):
    intext=re.sub(regex1, ' ', intext)
    intext=re.sub(regex2, ' ', intext)
    intext=re.sub(regex3, ' ', intext)
    
    #tokenize sentence
    dirty_tokens = [word.lower() for sent in nltk.sent_tokenize(intext) for word in nltk.word_tokenize(sent)]
    tokens=[]
    for eacht in dirty_tokens:
        if eacht not in punctuation:
            tokens.append(eacht)
    stemmed_tokens = [stemmer.stem(t) for t in tokens]
    
    return " ".join(stemmed_tokens)
    
    


In [35]:
print clean_and_tokenize("The bike was locked to a sign pole by the corner of the superstore. The lock was a combination lock and i had \
tied chained my helmet to the bike. The bike was of make 'GT' with shimano gears and had a custom seat. The seat had a ^ symbol \
and the spokes had multiple $ reflectors")

the bike was lock to a sign pole by the corner of the superstor the lock was a combin lock and i had tie chain my helmet to the bike the bike was of make gt ' with shimano gear and had a custom seat the seat had a symbol and the spoke had multipl reflector


In [38]:
bike_subdf['Clean_theft_description'] = bike_subdf.theft_description.map(clean_and_tokenize)
bike_subdf.to_csv(DATAFILEPATH+'stolen_bikes_tokenized.csv',encoding='utf-8')

/Users/Dilip_MBP/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [45]:
#Tokenize each theft description to return a list of nouns, proper nouns etc for each sample (row)
# We will remove words whose word count is less than 2
vectorizer = CountVectorizer(min_df=2, stop_words='english')
text_sparse_matrix = vectorizer.fit_transform(bike_subdf.Clean_theft_description)    #************* May need to use mask for train/test 
id2words= dict((v, k) for k, v in vectorizer.vocabulary_.iteritems())
corpus_gensim = gensim.matutils.Sparse2Corpus(text_sparse_matrix, documents_columns=False)

In [46]:
print "Corpus shape:", corpus_gensim.sparse.get_shape()
print "Number of words in vocabulary:", len(vectorizer.vocabulary_)


Corpus shape: (4949, 29838)
Number of words in vocabulary: 4949


In [52]:
#Use sparse matrix to initiate K-means clustering


from sklearn.cluster import KMeans

num_clusters = np.arange(2,11)
kmeans_model={}

for i in num_clusters:
    kmeans_model[i] = KMeans(init='k-means++', n_clusters=i, n_init=10)
    %time kmeans_model[i].fit(text_sparse_matrix)



CPU times: user 14.6 s, sys: 11.4 ms, total: 14.6 s
Wall time: 14.6 s
CPU times: user 19.2 s, sys: 8.36 ms, total: 19.2 s
Wall time: 19.2 s
CPU times: user 15.9 s, sys: 9.95 ms, total: 15.9 s
Wall time: 15.9 s
CPU times: user 17.5 s, sys: 12 ms, total: 17.5 s
Wall time: 17.5 s
CPU times: user 22.1 s, sys: 16.1 ms, total: 22.1 s
Wall time: 22.1 s
CPU times: user 20.1 s, sys: 22.2 ms, total: 20.1 s
Wall time: 20.1 s
CPU times: user 25.3 s, sys: 22 ms, total: 25.3 s
Wall time: 25.3 s
CPU times: user 26.2 s, sys: 13.3 ms, total: 26.2 s
Wall time: 26.2 s
CPU times: user 18.2 s, sys: 12.4 ms, total: 18.2 s
Wall time: 18.2 s


In [54]:
for i in num_clusters:
    print "Printing cluster centers for Kmeans with ", i, " of clusters"
    print kmeans_model[i].cluster_centers_

Printing cluster centers for Kmeans with  2  of clusters
[[  8.32039386e-02   9.84661996e-04   7.57432304e-05 ...,   1.13614846e-04
    1.13614846e-04   1.89358076e-04]
 [  3.40809787e-02   5.82580833e-04   0.00000000e+00 ...,   8.73871250e-04
    0.00000000e+00   0.00000000e+00]]
Printing cluster centers for Kmeans with  3  of clusters
[[  5.68561873e-03   0.00000000e+00   0.00000000e+00 ...,   8.36120401e-05
    8.36120401e-05   1.67224080e-04]
 [  1.43103448e-01   1.72413793e-03   1.32625995e-04 ...,   1.32625995e-04
    1.32625995e-04   1.98938992e-04]
 [  3.14510365e-02   7.14796283e-04   0.00000000e+00 ...,   1.07219442e-03
    0.00000000e+00   0.00000000e+00]]
Printing cluster centers for Kmeans with  4  of clusters
[[  5.65513275e-03   0.00000000e+00   0.00000000e+00 ...,   9.58497077e-05
    9.58497077e-05   1.91699415e-04]
 [  7.97709924e-02   1.33587786e-03   0.00000000e+00 ...,   1.90839695e-04
    0.00000000e+00   0.00000000e+00]
 [  3.25274725e-02   8.79120879e-04   0.000